**If you lost points on the last checkpoint you can get them back by responding to TA/IA feedback**  

Update/change the relevant sections where you lost those points, make sure you respond on GitHub Issues to your TA/IA to call their attention to the changes you made here.

Please update your Timeline... no battle plan survives contact with the enemy, so make sure we understand how your plans have changed.

# COGS 108 - Data Checkpoint

# Names

- Carmen Truong
- Rabih Siddiqui
- Lewis Weng
- Jacob Lin
- Zifan Luo

# Research Question

Do restaurants further from college/university campuses have higher or lower ratings than those that are closer? Can we determine if the reviewers of these restaurants are students and if so, do students increase or lower Yelp ratings at these restaurants?

## Background and Prior Work

Colleges and universities host many students every year and as the number of students grow business around these places also grow, some even forming college towns where businesses’ livelihoods depend on students like at UC Davis. We were curious as to specifically restaurants in the vicinity of a college or university and as students of UCSD, it follows that we were curious about restaurants around that specific area. As a restaurant’s success or profits are hard to measure and obtain, we settled on trying to see if restaurants’ ratings are affected by their vicinity to UCSD.

One article published by QSR Magazine seems to indicate so, citing data from “College & University Keynote Report” from Datassential saying that 58% of students eat off campus and 49% of students consider themselves foodies and are more conscious about what they eat.<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) This means that more than half of the student body is regularly eating at restaurants around their campus and reviewing and recommending those restaurants. Another study that more closely looks at customer satisfaction with food service, points out several important factors that can contribute to a restaurant’s rating such as their food quality, service quality, decor quality, and most importantly price.<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2) The conclusions they found is that good service and then good food were the best indicators of a high customer satisfaction. These point out significant attributes that aren’t considered in our research question, and possibly could be much greater of a factor of a restaurant’s rating than distance.

1. Baltazar, Amanda. “Restaurants Would Be Wise to Court College Students.” QSR Magazine, 7 July 2023, https://www.qsrmagazine.com/operations/business-advice/restaurants-would-be-wise-court-college-students/
2. Serhan, Mireille, and Carole Serhan. “The Impact of Food Service Attributes on Customer Satisfaction in a Rural University Campus Environment.” International Journal of Food Science, Hindawi, 31 Dec. 2019, https://www.hindawi.com/journals/ijfs/2019/2154548/

# Hypothesis


We believe that restaurants further from the UCSD campus will have similar ratings as those closer to UCSD campus. While an influx of students in the area can have an impact on nearby restaurants, we believe that this impact will be minimal and negligible. This is because students don’t make up a majority of a restaurants’ clientele, at least around UCSD. There are many other customers that either live in the area or are traveling that can give ratings to restaurants.

# Data

## Data overview


Dataset #1  
- Dataset Name: Yelp Academic Dataset
- Link to the dataset: https://www.yelp.com/dataset
- Number of observations: 6990280 reviews, 150346 businesses
- Number of variables: 9, 14

This dataset contains information on a selection of businesses, reviews, and user data centered around different metropolitan areas from the app Yelp. It is separated into 5 different json files, businesses, reviews, checkin, tip, user. We only particularly care about the businesses and reviews. For businesses, the variables we care about are the business_id, city, longitude, latitude, stars, review_count. For reviews, the variables we care about are the business_id, stars and text.

## Yelp Academic Dataset

In [1]:
# import necessary libraries

import numpy as np
import pandas as pd

First let's load the businesses into a dataframe.

In [2]:
# load business data
business = pd.read_json('https://drive.usercontent.google.com/download?id=1HGtRB3g1Hx1t1j2vPqCdTEfG-WJtTFVN&confirm=xxx', lines=True)

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


Let's drop all the observations with missing values in the important columns

In [4]:
business = business.dropna(subset=['latitude', 'longitude', 'stars', 'review_count', 'categories']) # this changes nothing though

Restaurants and cafes are the businesses that we care about, so let's filter our business dataframe by category.

In [5]:
def identify_restaurants(data, keywords):
    keywords = [keyword.lower() for keyword in keywords]
    def check_categories(category):
        return any(keyword in category.lower() for keyword in keywords)
    return data[data['categories'].apply(check_categories)]

In [6]:
keywords = ['Restaurants', 'Food', 'Coffee & Tea']
identify_restaurants(business, keywords)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,1,"{'BusinessParking': '{'garage': False, 'street...","Specialty Food, Food, Coffee & Tea, Coffee Roa...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-14:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [7]:
d = business['city'].value_counts()
d[:30]

city
Philadelphia        14560
Tucson               9244
Tampa                9042
Indianapolis         7537
Nashville            6969
New Orleans          6208
Reno                 5929
Edmonton             5053
Saint Louis          4824
Santa Barbara        3828
Boise                2934
Clearwater           2221
Saint Petersburg     1660
Metairie             1639
Sparks               1621
Wilmington           1444
Franklin             1414
St. Louis            1255
St. Petersburg       1185
Meridian             1043
Brandon              1032
Largo                1000
Carmel                966
Cherry Hill           960
West Chester          838
Goleta                798
Brentwood             767
Palm Harbor           665
Greenwood             649
New Port Richey       603
Name: count, dtype: int64

From above we can see that the dataset has data for restaurants from many different cities. We will pick a few and find universities/colleges within to base our research around. Our initial selection was based on the top cities (with most reviews) in our dataset. However, upon further analysis, we identified that 51% of the cities in the top 29 in our dataset didn't have universities that fit our research criteria. We then refined our list to only include cities with universities in the city limits.

In [8]:
# dictionary of city name to a list of (university name, long, lat)
# universities = {'Philadelphia': [('University of Pennsylvania', 39.952305, -75.193703)]}
universities = pd.read_csv('./yelp_dataset/universities.csv')

In [9]:
universities = universities.set_index('City')

In [10]:
universities

,Country,State,University Name,Latitude,Longitude
City,,,,,
Philidelphia,USA,Pennsylvania,University of Pennsylvania,39.952583,-75.191975
Tuscon,USA,Arizona,University of Arizona Tuscon,32.221664,-110.948922
Tampa,USA,Florida,University of South Florida,28.051836,-82.400005
Indianapolis,USA,Indiana,Purdue University,40.422902,-86.901785
Nashville,USA,Tennessee,Vanderbilt University,36.145532,-86.804060
New Orleans,USA,Louisiana,Tulane University,29.958586,-90.064997
Reno,USA,Nevada,University of Nevada,39.534642,-119.812831
Edmonton,Canada,Alberta,University of Alberta,53.523219,-113.523219
St. Louis,USA,Missouri,Washington University in St. Louis,38.628900,-90.307200


Let's also filter our business dataframe to only include businesses inside these cities.

In [11]:
business = business[business['city'].isin(['Philidelphia', 'Tuscon', 'Tampa', 'Indianapolis', 'Nashville',
       'New Orleans', 'Reno', 'Edmonton', 'St. Louis', 'Santa Barbara'])]

We can then use this information on universities to calculate whether a business is close or far from a university using their latitude and longitude positions.

In [12]:
# this is a lat long distance calculator from https://community.esri.com/t5/coordinate-reference-systems-blog/distance-on-a-sphere-the-haversine-formula/ba-p/902128#:~:text=All%20of%20these%20can%20be,longitude%20of%20the%20two%20points

def haversine(coord1, coord2):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0)**2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    km = round(km, 3)


#     print(f"Distance: {meters} m")
#     print(f"Distance: {km} km‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍")
    return km

In [13]:
def calc_distance(df):
    new_df = df.copy()
    lat, long = universities.loc[df['city'], ['Latitude', 'Longitude']]
    lat1, long1 = df[['latitude', 'longitude']]
    # threshold 10 km
    new_df['close_to_university'] = haversine((lat, long), (lat1, long1)) < 10
    return new_df

We decided to use a threshold value of 10 km. A business would be considered close to an university/college if it is within 10 km of one and far otherwise.

In [14]:
business = business.apply(calc_distance, axis=1)

Now let's create a dataframe for our reviews. There are simply too many reviews (6990280!) and the kernel cannot handle loading that many observations into memory, thus we have cut down the file using python to just the first 100000 reviews.

In [15]:
review = pd.read_json('https://drive.usercontent.google.com/download?id=1xE5dbDWd1Mp8kFQwoMmtLVj5xPq9tpuG&confirm=xxx', lines=True)

In [16]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [17]:
review.dropna(subset=['stars', 'text'])

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
99995,pAEbIxvr6ebx2bHc1XvguA,SMH5CeiLvKx61lKwtLZ_PA,lV0k3BnslFRkuWD_kbKd0Q,4,0,0,0,Came here for lunch with a group. They were bu...,2018-05-30 22:28:56
99996,xH1AoE-4nf2ECGQJRjO4_g,2clTdtp-BjphxLjN83CpUA,G0xz3kyRhRi6oZl7KfR0pA,1,1,0,0,The equipment is so old and so felty! I just u...,2015-04-05 23:31:52
99997,GatIbXTz-WDru5emONUSIg,MRrN6DH3QGCFcDv5RENYVg,C4lZdhasjZVQyDlOiXY1sA,4,0,0,0,This is one of my favorite Mexican restaurants...,2016-06-04 00:59:15
99998,6NfkodAdhvI89xONXuBC3A,rnNQzeKJbvqVCsYsL10mkQ,dChRGpit9fM_kZK5pafNyA,2,0,0,0,Came here for brunch - had an omlette ($19 + t...,2018-06-11 12:45:08


To figure out which reviews are written by college students, we queried reviews that have any mention of words related to them. To do this, we chose keywords such as student, students, college, colleges, university, universities, and uni. These keywords were chosen in the hopes that the review was written by a person who is mentioning their status as a student to be used as their way of reliability to other reviewees.

We also want to disclose concerns of using such a method. By querying reviews written by students this way, we could be counting false positives and missing false negatives. In this case, a false positive is when a review incorrectly classifies a non-student's review as a student, while a false negative is when a review incorrectly classifies a student's review as a non-student. Here are a few examples:
1. False Positive: A non-student brought up a "college" nearby.
2. False Positive: The "students" mentioned in a review could be students in highschool, not college.
3. False Positive: A non-student talks about how a restaurant is often visited by many "students"
4. False Negative: The review could have been written by a student, but the person did not mention that they were a student in their review.

In [18]:
def identify_student_reviews(data, keywords):

    keywords = [keyword.lower() for keyword in keywords]


    def check_keywords(review):

        return any(keyword in review.lower() for keyword in keywords)

    data['student_or_not'] = data['text'].apply(check_keywords)
    return data

In [19]:
keywords = ['student', 'students', 'college', 'colleges', 'university', 'universities', 'uni']
review_student = identify_student_reviews(review, keywords)

In [20]:
review_student.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,student_or_not
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,False
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,False
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,False
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,False
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,False


We can then inner join these two dataframe's on business_id to get a dataframe where the observations are reviews with information on if the review was written by a student and if the business the review was written for is close or far from a university/college.

In [21]:
review_businesses = pd.merge(review_student, business, how='inner', on='business_id')

Let's reduce this dataframe to the columns that we care about, namely the review rating, whether its written by a student, business' average rating, the amount of reviews that business has, and whether that business is close or far from a university/college.

In [22]:
review_businesses = review_businesses[['name', 'stars_x', 'student_or_not', 'stars_y', 'review_count', 'close_to_university']]
review_businesses = review_businesses.rename(columns={'name': 'restaurant_name', 'stars_x': 'rating', 'stars_y': 'avg_rating'})

In [23]:
review_businesses

,restaurant_name,rating,student_or_not,avg_rating,review_count,close_to_university
0,Melt,4,False,4.0,32,True
1,Melt,5,False,4.0,32,True
2,Melt,5,False,4.0,32,True
3,Melt,5,False,4.0,32,True
4,Melt,5,False,4.0,32,True
...,...,...,...,...,...,...
39257,Grand China,5,False,3.5,19,False
39258,RNR Tire Express & Custom Wheels,1,False,2.5,5,True
39259,"Rook Thomas, DC",5,False,4.0,6,False
39260,Chaps Party Rental,5,False,3.0,5,False


# Ethics & Privacy

We need inclusive data representation, by ensuring the dataset adequately represents different types of restaurants and doesn’t exclude any significant group. We must ensure that the data sources that we use comply with ethical standards and privacy laws. The data should be publicly available and not include any personal information about individuals who submitted the ratings. We must review the data and results to monitor for biases.

The data on Yelp and Google Maps is self-uploaded, so some restaurants may not appear. Since the data is self-uploaded, we believe the restaurants would be open to having other people see it. The restaurants are self-uploaded. After a restaurant is uploaded, they cannot be taken down unless the restaurant has closed down. Since closed restaurants do not show up, the data could be biased. For instance, restaurants that have closed down could potentially have lower ratings, but these lower ratings are no longer part of the data. Another bias could be with online reviews in general. Restaurants could encourage positive reviews by offering a discount or a free dessert to customers. Another concern is that people who have a negative experience might feel frustrated and post negative reviews while a person who has a great experience has no problems, so they also don’t feel the need to make a review. Negative reviews may be overrepresented. 

We must develop a methodology that addresses these issues, such as figuring out how we are going to include each level of budget, restaurant type, etc. We have to be sure to exclude personal identifiers (such as names) for those who submitted reviews. We also have to make sure to remove the reviewer’s names. We will include restaurants in the main price categories and all cuisines that are ordered to avoid bias.

# Team Expectations 

* Team members are expected to attend regular discord meetings, contribute ideas and actively participate in project discussions.
* Team members should be communicative and complete their assigned works by the agreed-upon deadlines.
* Team members should provide feedback to each other’s work in a respectful manner.

# Project Timeline Proposal

Specify your team's specific project timeline. An example timeline has been provided. Changes the dates, times, names, and details to fit your group's plan.

If you think you will need any special resources or training outside what we have covered in COGS 108 to solve your problem, then your proposal should state these clearly. For example, if you have selected a problem that involves implementing multiple neural networks, please state this so we can make sure you know what you’re doing and so we can point you to resources you will need to implement your project. Note that you are not required to use outside methods.



| Meeting Date   | Meeting Time    | Completed Before Meeting                                                         | Discuss at Meeting                                                       |
|:---------------|:----------------|:--------------------------------------------------------------------------------|:--------------------------------------------------------------------------|
| 1/20           | 1 PM            | Read & Think about COGS 108 expectations; brainstorm topics/questions       | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research |
| 1/26           | 10 AM           | Do background research on topic                                                | Discuss ideal dataset(s) and ethics; draft project proposal              |
| 2/1            | 10 AM           | Edit, finalize, and submit proposal; Search for datasets                     | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part |
| 2/14           | 6 PM            | Gather and Import Data (Jacob, Rabih); Wrangle and Preprocess Data (Lewis, Carmen, Zifan) |                                                                           |
| 2/23           | 12 PM           | Finalize wrangling/EDA; Begin Analysis (Iron Man)                             | Discuss/edit Analysis; Complete project check-in                        |
| 2/25           | 2:30 PM         | Complete Data Checkpoint                                                       |                                                                           |
| 3/13           | 12 PM           | Complete analysis; Draft results/conclusion/discussion (Wasp)                  | Discuss/edit full project                                                |
| 3/20           | Before 11:59 PM | NA                                                                            | Turn in Final Project & Group Project Surveys                            |